In [12]:
import os
import torch
import pytorch_lightning as L
import matplotlib.pyplot as plt
from omegaconf import DictConfig

%matplotlib inline
plt.rcParams['image.interpolation'] = 'nearest'

%load_ext autoreload
%autoreload 2

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
print(torch.cuda.get_device_properties(0).total_memory)

# Get number of cores
print(os.cpu_count())

# Get number of threads
print(torch.get_num_threads())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
True
Tesla V100-SXM2-32GB
34072559616
72
36


In [15]:
import sklearn  # scikit-learn hack to fix the error on jetson

import torch
import hydra
import numpy as np
from PIL import Image
import albumentations as A
import pytorch_lightning as L
import matplotlib.pyplot as plt
from omegaconf import DictConfig, OmegaConf
from albumentations.pytorch import ToTensorV2
from pytorch_lightning.loggers import WandbLogger

from src import RoadDataModule, RoadModel, LogPredictionsCallback, val_checkpoint, regular_checkpoint, rgb_to_label


def main(cfg: DictConfig) -> None:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = RoadModel(cfg, device)
    datamodule = RoadDataModule(cfg)

    wandb_logger = WandbLogger(project="road-segmentation", name=cfg.run_name)

    trainer = L.Trainer(max_epochs=cfg.train.max_epochs,
                        accelerator="gpu",
                        devices=1,
                        logger=wandb_logger,
                        callbacks=[
                            LogPredictionsCallback(),
                            val_checkpoint,
                            regular_checkpoint
                        ])

    if cfg.action == "train":
        trainer.fit(model, datamodule=datamodule, ckpt_path=cfg.ckpt_path)
    elif cfg.action == "test":
        trainer.test(model, datamodule=datamodule, ckpt_path=cfg.ckpt_path)
    elif cfg.action == "predict":
        # Load the trained model
        model = RoadModel.load_from_checkpoint(cfg.ckpt_path, cfg=cfg, device=device).to(device)
        model.eval()

        # Load an image and its label
        # image_path = 'data/RUGD/Images/creek_00001.png'
        # label_path = 'data/RUGD/Annotations/creek_00001.png'        
        
        image_path = 'data/RoboTour/Images/tradr_20_5_2422.png'
        label_path = 'data/RoboTour/Annotations/tradr_20_5_2422.png'

        image = np.array(Image.open(image_path).convert('RGB')) / 255.0
        label = np.array(Image.open(label_path).convert('RGB'))

        # Process the label image
        label = rgb_to_label(label, cfg.ds.color_map)
        train_map = OmegaConf.to_container(cfg.ds.train_map)
        label = np.vectorize(train_map.get)(label)

        # Apply the same transformations as during training
        transform = A.Compose([
            A.Normalize(mean=cfg.ds.mean, std=cfg.ds.std, max_pixel_value=1.0),
            A.Resize(550, 688),
            ToTensorV2()
        ])
        sample = transform(image=image, mask=label)
        image = sample['image'].float().unsqueeze(0).to(device)
        label = sample['mask'].long().unsqueeze(0).to(device)

        # Predict the label image
        with torch.no_grad():
            logits = model(image)
        prediction = logits.argmax(1).squeeze(0).cpu().numpy()

        # Plot the image, label, and prediction
        fig = plt.figure(figsize=(12, 4))

        # Plot the prediction next to the label
        plt.subplot(1, 3, 1)
        plt.imshow(image[0].permute(1, 2, 0).cpu().numpy())
        plt.axis('off')  # Remove axes

        plt.subplot(1, 3, 2)
        plt.imshow(label[0].cpu().numpy())
        plt.axis('off')  # Remove axes

        plt.subplot(1, 3, 3)
        plt.imshow(prediction)
        plt.axis('off')  # Remove axes

        plt.subplots_adjust(wspace=0, hspace=0)
        plt.tight_layout()

        # Save the plot
        plt.savefig('prediction.png')
    else:
        raise ValueError(f"Unknown action: {cfg.action}")

In [19]:
from hydra import compose, initialize

with initialize(version_base=None, config_path="conf"):
    # Make sure that the cfg.ds is 'robotour' 
    config = compose(config_name="config", overrides=["action=train", "ds=robotour", "ckpt_path=checkpoints/e51-iou0.60.ckpt"])
    print(config)
    main(config)

{'action': 'train', 'ckpt_path': 'checkpoints/e51-iou0.60.ckpt', 'run_name': 'baseline', 'optimizer': {'_target_': 'torch.optim.Adam', 'lr': 0.0001, 'weight_decay': 0.0001}, 'train': {'batch_size': 4, 'max_epochs': 100, 'num_workers': 4}, 'ds': {'name': 'rugd', 'path': 'data/RoboTour', 'mean': [0.0, 0.0, 0.0], 'std': [1.0, 1.0, 1.0], 'color_map': {'0,0,0': 0, '0,255,0': 1, '255,0,0': 2, '0,0,255': 3}, 'train_map': {0: 0, 1: 1, 2: 2, 3: 3}}, 'model': {'_target_': 'torchvision.models.segmentation.deeplabv3_resnet50', 'num_classes': 4}}


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Restoring states from the checkpoint path at checkpoints/e51-iou0.60.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                   | Params
-----------------------------------------------------
0 | model     | DeepLabV3              | 39.6 M
1 | criterion | CrossEntropyLoss       | 0     
2 | accuracy  | MulticlassAccuracy     | 0     
3 | jaccard   | MulticlassJaccardIndex | 0     
-----------------------------------------------------
39.6 M    Trainable params
0         Non-trainable params
39.6 M    Total params
158.538   Total estimated model params size (MB)
Restored all states from the checkpoint at checkpoints/e51-iou0.60.ckpt
SLURM auto-requeueing enabled. Setting signal handlers.


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

wandb: WARNING (User provided step: 51 is less than current step: 119. Dropping entry: {'prediction': {'_type': 'image-file', 'sha256': '85e0251e5dbb4f7ed46382dffe961d6138ea6c02ae35ff9f5141bdf42d76bf94', 'size': 440534, 'path': 'media/images/prediction_119_85e0251e5dbb4f7ed463.png', 'format': 'png', 'width': 688, 'height': 550, 'caption': 'Prediction', 'masks': {'prediction': {'_type': 'mask', 'sha256': 'cda292833009710e757d52422f4408fa1910ed9b8d9e8d38419003dc3ee7941e', 'size': 4127, 'path': 'media/images/mask/prediction_119_cda292833009710e757d.mask.png'}}}, '_timestamp': 1716384779.7776153}).
wandb: WARNING (User provided step: 51 is less than current step: 119. Dropping entry: {'entropy': {'_type': 'image-file', 'sha256': '3bb7c72964b50cfc0b46a9362d99060d995e29f8a82c90fe629e7e6f91a9cf2b', 'size': 169658, 'path': 'media/images/entropy_119_3bb7c72964b50cfc0b46.png', 'format': 'png', 'width': 688, 'height': 550, 'caption': 'Entropy'}, '_timestamp': 1716384779.9026396}).
wandb: WARNING 

...

wandb: WARNING (User provided step: 97 is less than current step: 261. Dropping entry: {'prediction': {'_type': 'image-file', 'sha256': '85e0251e5dbb4f7ed46382dffe961d6138ea6c02ae35ff9f5141bdf42d76bf94', 'size': 440534, 'path': 'media/images/prediction_261_85e0251e5dbb4f7ed463.png', 'format': 'png', 'width': 688, 'height': 550, 'caption': 'Prediction', 'masks': {'prediction': {'_type': 'mask', 'sha256': 'c7ec32fd74413793128523d1558a529992f75a5d525fd19b92fe1de4bcf37b92', 'size': 1756, 'path': 'media/images/mask/prediction_261_c7ec32fd744137931285.mask.png'}}}, '_timestamp': 1716385648.4047794}).
wandb: WARNING (User provided step: 97 is less than current step: 261. Dropping entry: {'entropy': {'_type': 'image-file', 'sha256': 'fc0fc94e60bb82cf4751ac0e2f1a51b2cd46661ade0992f95fad70dad2f458fd', 'size': 42099, 'path': 'media/images/entropy_261_fc0fc94e60bb82cf4751.png', 'format': 'png', 'width': 688, 'height': 550, 'caption': 'Entropy'}, '_timestamp': 1716385648.4728444}).
wandb: WARNING (

Validation: |          | 0/? [00:00<?, ?it/s]

wandb: WARNING (User provided step: 98 is less than current step: 264. Dropping entry: {'prediction': {'_type': 'image-file', 'sha256': '85e0251e5dbb4f7ed46382dffe961d6138ea6c02ae35ff9f5141bdf42d76bf94', 'size': 440534, 'path': 'media/images/prediction_264_85e0251e5dbb4f7ed463.png', 'format': 'png', 'width': 688, 'height': 550, 'caption': 'Prediction', 'masks': {'prediction': {'_type': 'mask', 'sha256': '2470d1888ff5098ea002eb18880a4d32c9df4a08e9a1759b1f1288b7ae37cead', 'size': 1769, 'path': 'media/images/mask/prediction_264_2470d1888ff5098ea002.mask.png'}}}, '_timestamp': 1716385667.7987213}).
wandb: WARNING (User provided step: 98 is less than current step: 264. Dropping entry: {'entropy': {'_type': 'image-file', 'sha256': '61877a242ec33a965836ba7d5167919ca0d723799180c178ec8d89170f8ce038', 'size': 42437, 'path': 'media/images/entropy_264_61877a242ec33a965836.png', 'format': 'png', 'width': 688, 'height': 550, 'caption': 'Entropy'}, '_timestamp': 1716385667.8680305}).
wandb: WARNING (

Validation: |          | 0/? [00:00<?, ?it/s]

wandb: WARNING (User provided step: 99 is less than current step: 267. Dropping entry: {'prediction': {'_type': 'image-file', 'sha256': '85e0251e5dbb4f7ed46382dffe961d6138ea6c02ae35ff9f5141bdf42d76bf94', 'size': 440534, 'path': 'media/images/prediction_267_85e0251e5dbb4f7ed463.png', 'format': 'png', 'width': 688, 'height': 550, 'caption': 'Prediction', 'masks': {'prediction': {'_type': 'mask', 'sha256': '72707cb3de4e6ff87eba96e7b633ab78058e1880b3df254b4cc08dc8794c5330', 'size': 1738, 'path': 'media/images/mask/prediction_267_72707cb3de4e6ff87eba.mask.png'}}}, '_timestamp': 1716385687.917018}).
wandb: WARNING (User provided step: 99 is less than current step: 267. Dropping entry: {'entropy': {'_type': 'image-file', 'sha256': 'dc36548bed9120e85751836f4f1d22630cc7c08024a0ae3cc0c3630e4fed34ba', 'size': 41356, 'path': 'media/images/entropy_267_dc36548bed9120e85751.png', 'format': 'png', 'width': 688, 'height': 550, 'caption': 'Entropy'}, '_timestamp': 1716385687.9845412}).
wandb: WARNING (U